In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install langchain_chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install -qU langchain-openai

In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# import dotenv

# dotenv.load_dotenv()

··········


In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

## With Conversation History

In [4]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

file_path = '/content/RAG Input Doc.pdf'  # Replace with the actual file path

# Create a FileLoader instance for the local file
loader = PyPDFLoader(file_path)

# Load the data from the file
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [5]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [6]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [7]:

q1 = "Which paper received the highest number of stars per hour?"
q2 = "What is the focus of the 'MeshAnything' project?"
q3 = "Which paper discusses integration of LLMs with Monte Carlo Tree Search?"
q4 = "What advancements does the 'VideoLLaMA 2' paper propose?"
q5 = "Which paper was published most recently?"
q6 = "Identify a paper that deals with language modeling and its scalability."
q7 = "Which paper aims at improving accuracy in Google-Proof Question Answering?"
q8 = '''List the categories covered by the paper titled 'TextGrad: Automatic "Differentiation" via
Text'. '''


In [8]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []


ai_msg_1 = rag_chain.invoke({"input": q1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=q1),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

print("############")
print(q1)
print(ai_msg_1["answer"])
print("############")


############
Which paper received the highest number of stars per hour?
The paper "MeshAnything: Artist-Created Mesh Generation with Autoregressive Transformers" received the highest number of stars per hour, with a rate of 5.09 stars per hour.
############


In [9]:
ai_msg_2 = rag_chain.invoke({"input": q2, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=q2),
        AIMessage(content=ai_msg_2["answer"]),
    ]
)

print(q2)
print(ai_msg_2["answer"])
print("############")


What is the focus of the 'MeshAnything' project?
The "MeshAnything" project focuses on artist-created mesh generation using autoregressive transformers. It aims to showcase the potential for replacing manually crafted assets with 3D assets generated via reconstruction and generation techniques.
############


In [10]:
ai_msg_3 = rag_chain.invoke({"input": q3, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=q3),
        AIMessage(content=ai_msg_3["answer"]),
    ]
)
print(q3)
print(ai_msg_3["answer"])
print("############")


Which paper discusses integration of LLMs with Monte Carlo Tree Search?
The paper "Accessing GPT-4 level Mathematical Olympiad Solutions via Monte Carlo Tree Self-refine with LLaMa-3 8B" discusses the integration of Large Language Models (LLMs) with Monte Carlo Tree Search (MCTS) through the MCT Self-Refine algorithm for enhancing performance in complex mathematical reasoning tasks.
############


In [11]:
ai_msg_4 = rag_chain.invoke({"input": q4, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=q4),
        AIMessage(content=ai_msg_4["answer"]),
    ]
)
print(q4)
print(ai_msg_4["answer"])
print("############")

What advancements does the 'VideoLLaMA 2' paper propose?
The "VideoLLaMA 2" paper proposes a set of Video Large Language Models (Video-LLMs) focused on enhancing spatial-temporal modeling and audio understanding in video and audio-oriented tasks. It aims to advance the capabilities of these models in handling complex video and audio data for improved performance in various tasks.
############


In [12]:
ai_msg_5 = rag_chain.invoke({"input": q5, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=q5),
        AIMessage(content=ai_msg_5["answer"]),
    ]
)

print(q5)
print(ai_msg_5["answer"])
print("############")

Which paper was published most recently?
The most recently published paper is "MeshAnything: Artist-Created Mesh Generation with Autoregressive Transformers," published on June 14, 2024.
############


In [13]:
ai_msg_6 = rag_chain.invoke({"input": q6, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=q6),
        AIMessage(content=ai_msg_6["answer"]),
    ]
)

print(q6)
print(ai_msg_6["answer"])
print("############")

Identify a paper that deals with language modeling and its scalability.
The paper "Scalable MatMul-free Language Modeling" addresses language modeling and scalability by demonstrating that MatMul-free models achieve comparable performance to state-of-the-art Transformers while requiring less memory during inference, particularly at scales up to at least 2.7B parameters.
############


In [14]:
ai_msg_7 = rag_chain.invoke({"input": q7, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=q7),
        AIMessage(content=ai_msg_7["answer"]),
    ]
)

print(q7)
print(ai_msg_7["answer"])
print("############")

Which paper aims at improving accuracy in Google-Proof Question Answering?
The paper "TextGrad: Automatic 'Differentiation' via Text" aims at improving the zero-shot accuracy of GPT-40 in Google-Proof Question Answering, among other tasks like optimizing LeetCode-Hard coding problem solutions and enhancing prompts for reasoning.
############


In [15]:
ai_msg_8 = rag_chain.invoke({"input": q8, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=q8),
        AIMessage(content=ai_msg_8["answer"]),
    ]
)
print(q8)
print(ai_msg_8["answer"])
print("############")

List the categories covered by the paper titled 'TextGrad: Automatic "Differentiation" via
Text'. 
The categories covered by the paper "TextGrad: Automatic 'Differentiation' via Text" are Decision Making and GSM8K +2.
############
